# ═══════════════════════════════════════════════════════════════════════════════
# PHASE 1: DATA EXPLORATION & ITEM DIAGNOSTICS
# ═══════════════════════════════════════════════════════════════════════════════

## 1.1 Environment Setup

In [ ]:
import sys
import warnings
import os

# Fix for potential OpenMP runtime conflicts
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
from IPython.display import display, Markdown

from factor_analyzer import FactorAnalyzer, calculate_kmo, calculate_bartlett_sphericity

warnings.filterwarnings('ignore')

# Ensure output directories exist
Path('plots').mkdir(exist_ok=True)
Path('tables').mkdir(exist_ok=True)

# Plot settings
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

RANDOM_SEED = 67
np.random.seed(RANDOM_SEED)

POPULATION = 'full'  # Using entire 511 sample for experimental analysis

print("✓ Libraries loaded")
print(f"✓ Random seed: {RANDOM_SEED}")
print(f"✓ Population: {POPULATION.upper()} (N=511)")

## 1.2 Load Data and Metadata

In [ ]:
# Load FULL sample for experimental EFA (N=511)
df_full = pd.read_csv('../data/AIRS_clean.csv')

# Load item metadata
with open('../data/airs_28item_complete.json', 'r', encoding='utf-8') as f:
    item_metadata = json.load(f)

# Extract structure - ALL items initially
ALL_PREDICTOR_ITEMS = item_metadata['predictor_items']  # 24 items
outcome_items = item_metadata['outcome_items']  # 4 BI items
ALL_CONSTRUCTS = item_metadata['constructs']
metadata = item_metadata['metadata']
positive_items = item_metadata['positive_items']
negative_items = item_metadata['negative_items']

# Create construct mapping dictionary
ALL_PREDICTOR_CONSTRUCTS = {
    'PE': ['PE1', 'PE2'],
    'EE': ['EE1', 'EE2'],
    'SI': ['SI1', 'SI2'],
    'FC': ['FC1', 'FC2'],
    'HM': ['HM1', 'HM2'],
    'PV': ['PV1', 'PV2'],
    'HB': ['HB1', 'HB2'],
    'VO': ['VO1', 'VO2'],
    'TR': ['TR1', 'TR2'],
    'EX': ['EX1', 'EX2'],
    'ER': ['ER1', 'ER2'],
    'AX': ['AX1', 'AX2'],
}

print(f"Full sample: N = {len(df_full)}")
print(f"Total predictor items: {len(ALL_PREDICTOR_ITEMS)}")
print(f"Total constructs: {len(ALL_PREDICTOR_CONSTRUCTS)}")
print(f"\nConstructs: {list(ALL_PREDICTOR_CONSTRUCTS.keys())}")

## 1.3 Descriptive Statistics - All Items

In [ ]:
# Descriptive statistics for ALL items
df_all_items = df_full[ALL_PREDICTOR_ITEMS].copy()

desc_stats = df_all_items.describe().T
desc_stats['skew'] = df_all_items.skew()
desc_stats['kurtosis'] = df_all_items.kurtosis()
desc_stats['construct'] = [metadata[item]['construct_abbr'] for item in desc_stats.index]

# Add item text for reference
desc_stats['item_text'] = [metadata[item]['question_text'][:50] + '...' for item in desc_stats.index]

print("DESCRIPTIVE STATISTICS - ALL 24 PREDICTOR ITEMS")
print("=" * 80)
display(desc_stats[['construct', 'mean', 'std', 'min', 'max', 'skew', 'kurtosis']].round(3))

# Flag potential issues
skew_issues = desc_stats[abs(desc_stats['skew']) > 1.0]
kurt_issues = desc_stats[abs(desc_stats['kurtosis']) > 3.0]

if len(skew_issues) > 0:
    print(f"\n⚠️ Items with |skewness| > 1.0: {list(skew_issues.index)}")
if len(kurt_issues) > 0:
    print(f"⚠️ Items with |kurtosis| > 3.0: {list(kurt_issues.index)}")

## 1.4 Item Reference Guide

Complete item text for informed selection decisions.

In [ ]:
# Display full item reference
print("=" * 90)
print("COMPLETE ITEM REFERENCE GUIDE")
print("=" * 90)

for construct, items in ALL_PREDICTOR_CONSTRUCTS.items():
    construct_name = metadata[items[0]]['construct']
    print(f"\n{'─' * 90}")
    print(f"📋 {construct}: {construct_name}")
    print(f"{'─' * 90}")
    for item in items:
        direction = metadata[item]['direction']
        text = metadata[item]['question_text']
        mean_val = df_all_items[item].mean()
        std_val = df_all_items[item].std()
        print(f"  {item} [{direction:8}]: \"{text}\"")
        print(f"           Mean={mean_val:.2f}, SD={std_val:.2f}")

## 1.5 Within-Construct Reliability Analysis

In [ ]:
# Within-construct analysis functions
def cronbach_alpha(data):
    """Calculate Cronbach's alpha for a set of items."""
    n_items = data.shape[1]
    if n_items < 2:
        return np.nan
    item_vars = data.var(axis=0, ddof=1)
    total_var = data.sum(axis=1).var(ddof=1)
    return (n_items / (n_items - 1)) * (1 - item_vars.sum() / total_var)

def item_total_correlation(df, items):
    """Calculate corrected item-total correlations."""
    results = {}
    total = df[items].sum(axis=1)
    for item in items:
        # Corrected: exclude item from total
        corrected_total = total - df[item]
        results[item] = df[item].corr(corrected_total)
    return results

# Analyze each construct
construct_diagnostics = []

print("=" * 90)
print("WITHIN-CONSTRUCT RELIABILITY ANALYSIS - ALL CONSTRUCTS")
print("=" * 90)

for construct, items in ALL_PREDICTOR_CONSTRUCTS.items():
    construct_name = metadata[items[0]]['construct']
    construct_df = df_all_items[items]
    
    # Inter-item correlation
    r = construct_df.corr().iloc[0, 1]
    
    # Cronbach's alpha
    alpha = cronbach_alpha(construct_df)
    
    # Item-total correlations
    itc = item_total_correlation(df_all_items, items)
    
    # Status assessment
    if alpha >= 0.70:
        status = "✓ Good"
    elif alpha >= 0.60:
        status = "~ Marginal"
    else:
        status = "✗ Poor"
    
    construct_diagnostics.append({
        'Construct': construct,
        'Name': construct_name,
        'Items': ', '.join(items),
        'r_inter': r,
        'Alpha': alpha,
        'ITC_1': itc[items[0]],
        'ITC_2': itc[items[1]],
        'Status': status
    })
    
    print(f"\n{construct} ({construct_name}):")
    print(f"  Inter-item r = {r:.3f}")
    print(f"  Cronbach's α = {alpha:.3f} {status}")
    print(f"  {items[0]} ITC = {itc[items[0]]:.3f}")
    print(f"  {items[1]} ITC = {itc[items[1]]:.3f}")

diagnostics_df = pd.DataFrame(construct_diagnostics)

In [ ]:
# Summary table
print("\n" + "=" * 90)
print("CONSTRUCT RELIABILITY SUMMARY")
print("=" * 90)

display(diagnostics_df[['Construct', 'Name', 'r_inter', 'Alpha', 'ITC_1', 'ITC_2', 'Status']].round(3))

# Classification counts
good = len(diagnostics_df[diagnostics_df['Alpha'] >= 0.70])
marginal = len(diagnostics_df[(diagnostics_df['Alpha'] >= 0.60) & (diagnostics_df['Alpha'] < 0.70)])
poor = len(diagnostics_df[diagnostics_df['Alpha'] < 0.60])

print(f"\n✓ Good reliability (α ≥ .70): {good}/12 constructs")
print(f"~ Marginal reliability (.60 ≤ α < .70): {marginal}/12 constructs")
print(f"✗ Poor reliability (α < .60): {poor}/12 constructs")

## 1.6 Within-Construct Correlation Heatmap

In [ ]:
# Heatmap visualization
fig, axes = plt.subplots(3, 4, figsize=(16, 12))
axes = axes.flatten()

for idx, (construct, items) in enumerate(ALL_PREDICTOR_CONSTRUCTS.items()):
    ax = axes[idx]
    construct_df = df_all_items[items]
    corr = construct_df.corr()
    
    alpha_val = diagnostics_df[diagnostics_df['Construct'] == construct]['Alpha'].values[0]
    status = diagnostics_df[diagnostics_df['Construct'] == construct]['Status'].values[0]
    
    color = 'darkgreen' if 'Good' in status else ('goldenrod' if 'Marginal' in status else 'darkred')
    
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='RdYlGn', center=0.5,
               vmin=0, vmax=1, ax=ax, cbar=False,
               xticklabels=items, yticklabels=items)
    ax.set_title(f"{construct} (α={alpha_val:.2f})", fontsize=10, color=color, fontweight='bold')

plt.suptitle('Within-Construct Inter-Item Correlations\n(Green title = Good α, Yellow = Marginal, Red = Poor)', 
             fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('plots/phase1_within_construct_correlations.png', dpi=150, bbox_inches='tight')
plt.show()

## 1.7 Item-Level Diagnostics Summary

In [ ]:
# Create comprehensive item diagnostics
item_diagnostics = []

for item in ALL_PREDICTOR_ITEMS:
    construct = metadata[item]['construct_abbr']
    direction = metadata[item]['direction']
    
    # Find ITC from construct analysis
    construct_items = ALL_PREDICTOR_CONSTRUCTS[construct]
    itc_dict = item_total_correlation(df_all_items, construct_items)
    
    # Item stats
    mean_val = df_all_items[item].mean()
    std_val = df_all_items[item].std()
    skew_val = df_all_items[item].skew()
    kurt_val = df_all_items[item].kurtosis()
    
    # Flags
    issues = []
    if abs(skew_val) > 1.0:
        issues.append('SKEW')
    if abs(kurt_val) > 3.0:
        issues.append('KURT')
    if itc_dict[item] < 0.30:
        issues.append('LOW_ITC')
    
    item_diagnostics.append({
        'Item': item,
        'Construct': construct,
        'Direction': direction,
        'Mean': mean_val,
        'SD': std_val,
        'Skew': skew_val,
        'Kurt': kurt_val,
        'ITC': itc_dict[item],
        'Issues': ', '.join(issues) if issues else '✓'
    })

item_diag_df = pd.DataFrame(item_diagnostics)

print("=" * 90)
print("ITEM-LEVEL DIAGNOSTICS SUMMARY")
print("=" * 90)
display(item_diag_df.round(3))

# Flag problem items
problem_items = item_diag_df[item_diag_df['Issues'] != '✓']
if len(problem_items) > 0:
    print(f"\n⚠️ ITEMS WITH POTENTIAL ISSUES ({len(problem_items)}):")
    for _, row in problem_items.iterrows():
        print(f"   {row['Item']} ({row['Construct']}): {row['Issues']}")
else:
    print("\n✓ All items pass basic diagnostics")

## 1.8 KMO and Bartlett's Test (All Items)

In [ ]:
# KMO and Bartlett's on ALL items
kmo_all, kmo_model = calculate_kmo(df_all_items)
chi_square, p_value = calculate_bartlett_sphericity(df_all_items)

print("=" * 90)
print("FACTORABILITY ASSESSMENT - ALL 24 ITEMS")
print("=" * 90)

print(f"\nKMO Measure of Sampling Adequacy: {kmo_model:.3f}")
if kmo_model >= 0.90:
    kmo_interpretation = "Marvelous"
elif kmo_model >= 0.80:
    kmo_interpretation = "Meritorious"
elif kmo_model >= 0.70:
    kmo_interpretation = "Middling"
elif kmo_model >= 0.60:
    kmo_interpretation = "Mediocre"
elif kmo_model >= 0.50:
    kmo_interpretation = "Miserable"
else:
    kmo_interpretation = "Unacceptable"
print(f"  Interpretation: {kmo_interpretation} (Kaiser, 1974)")

print(f"\nBartlett's Test of Sphericity:")
print(f"  Chi-square: {chi_square:.2f}")
print(f"  p-value: {p_value:.2e}")
print(f"  Result: {'✓ Significant' if p_value < 0.001 else '✗ Not significant'}")

# Item-level KMO
print(f"\nItem-level KMO (flagged if < 0.50):")
low_kmo_items = [(ALL_PREDICTOR_ITEMS[i], kmo_all[i]) for i in range(len(ALL_PREDICTOR_ITEMS)) if kmo_all[i] < 0.50]
if low_kmo_items:
    for item, kmo in low_kmo_items:
        print(f"  ⚠️ {item}: {kmo:.3f}")
else:
    print("  ✓ All items have KMO ≥ 0.50")

# ═══════════════════════════════════════════════════════════════════════════════
# 🎯 DECISION POINT 1: SELECT CONSTRUCTS AND ITEMS
# ═══════════════════════════════════════════════════════════════════════════════

Based on the diagnostics above, decide which items to include in the EFA.

**Considerations:**
- Constructs with poor reliability (α < .60) may not hold together
- Items with low ITC (< .30) may not contribute to their construct
- Items with extreme skewness or kurtosis may cause issues
- Theoretical relevance to your population

**Edit the `SELECTED_ITEMS` dictionary below to customize your selection.**

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  🎯 DECISION POINT 1: EDIT THIS DICTIONARY TO SELECT ITEMS                   ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

# ═══════════════════════════════════════════════════════════════════════════════
# ITEM REFERENCE (for cherry-picking decisions)
# ═══════════════════════════════════════════════════════════════════════════════
# PE - Performance Expectancy (POSITIVE)
#   PE1: "AI tools help me accomplish tasks more quickly."
#   PE2: "Using AI improves the quality of my work or studies."
#
# EE - Effort Expectancy (POSITIVE)
#   EE1: "Learning to use AI tools is easy for me."
#   EE2: "Interacting with AI tools is clear and understandable."
#
# SI - Social Influence (POSITIVE)
#   SI1: "People whose opinions I value encourage me to use AI tools."
#   SI2: "Leaders in my organization or school support the use of AI tools."
#
# FC - Facilitating Conditions (POSITIVE)
#   FC1: "I have access to training or tutorials for the AI tools I use."
#   FC2: "The AI tools I use are compatible with other tools or systems I use."
#
# HM - Hedonic Motivation (POSITIVE)
#   HM1: "Using AI tools is stimulating and engaging."
#   HM2: "AI tools make my work or studies more interesting."
#
# PV - Price Value (POSITIVE)
#   PV1: "I get more value from AI tools than the effort they require."
#   PV2: "Using AI tools is worth the learning curve."
#
# HB - Habit (POSITIVE)
#   HB1: "Using AI tools has become a habit for me."
#   HB2: "I tend to rely on AI tools by default when I need help with tasks."
#
# VO - Voluntariness (POSITIVE)
#   VO1: "I choose to use AI tools in my work because I find them helpful, not because I am required to."
#   VO2: "I could choose not to use AI tools in my work or studies if I preferred."
#
# TR - Trust in AI (POSITIVE)
#   TR1: "I trust AI tools to provide reliable information."
#   TR2: "I trust the AI tools that are available to me."
#
# EX - Explainability (POSITIVE)
#   EX1: "I understand how the AI tools I use generate their outputs."
#   EX2: "I prefer AI tools that explain their recommendations."
#
# ER - Perceived Ethical Risk (NEGATIVE)
#   ER1: "I worry that AI tools could replace jobs in my field."
#   ER2: "I am concerned about privacy risks when using AI tools."
#
# AX - AI Anxiety (MIXED)
#   AX1: "I feel uneasy about the increasing use of AI." (NEGATIVE)
#   AX2: "I worry that I may be left behind if I do not keep up with AI." (POSITIVE - FOMO)
# ═══════════════════════════════════════════════════════════════════════════════

SELECTED_ITEMS = {
    'PE': ['PE1', 'PE2'],  # Performance Expectancy
    'EE': ['EE1', 'EE2'],  # Effort Expectancy
    'SI': ['SI1', 'SI2'],  # Social Influence
    'FC': ['FC1', 'FC2'],  # Facilitating Conditions
    'HM': ['HM1', 'HM2'],  # Hedonic Motivation
    'PV': ['PV1', 'PV2'],  # Price Value
    'HB': ['HB1', 'HB2'],  # Habit
    'VO': ['VO1', 'VO2'],  # Voluntariness
    'TR': ['TR1', 'TR2'],  # Trust in AI
    'EX': ['EX1', 'EX2'],  # Explainability
    'ER': ['ER1', 'ER2'],  # Perceived Ethical Risk (NEGATIVE)
    'AX': ['AX1', 'AX2'],  # AI Anxiety (MIXED direction)
}

# ═══════════════════════════════════════════════════════════════════════════════
# Apply selection and update working variables
# ═══════════════════════════════════════════════════════════════════════════════

selected_predictor_items = [item for items in SELECTED_ITEMS.values() for item in items]
selected_predictor_constructs = {k: v for k, v in SELECTED_ITEMS.items() if v}

# Create EFA dataframe with selected items
df_efa = df_full[selected_predictor_items].copy()

# Update global references
predictor_items = selected_predictor_items
predictor_constructs = selected_predictor_constructs

# Summary
total_selected = len(predictor_items)
constructs_active = len([k for k, v in SELECTED_ITEMS.items() if v])
constructs_full = len([k for k, v in SELECTED_ITEMS.items() if len(v) == 2])
constructs_partial = len([k for k, v in SELECTED_ITEMS.items() if len(v) == 1])
constructs_dropped = len([k for k, v in SELECTED_ITEMS.items() if len(v) == 0])

print("=" * 90)
print("🎯 DECISION POINT 1 - ITEM SELECTION APPLIED")
print("=" * 90)
print(f"\n📊 Selection Summary:")
print(f"   Total items selected: {total_selected}/24")
print(f"   Active constructs: {constructs_active}/12")
print(f"     - Full (2 items): {constructs_full}")
print(f"     - Partial (1 item): {constructs_partial}")
print(f"     - Dropped (0 items): {constructs_dropped}")

print(f"\n📋 Selected Items by Construct:")
for construct, items in SELECTED_ITEMS.items():
    if items:
        print(f"   {construct}: {', '.join(items)}")
    else:
        print(f"   {construct}: [DROPPED]")

print(f"\n✓ df_efa ready: {df_efa.shape[0]} observations × {df_efa.shape[1]} items")

# ═══════════════════════════════════════════════════════════════════════════════
# PHASE 2: FACTOR EXPLORATION
# ═══════════════════════════════════════════════════════════════════════════════

## 2.1 Parallel Analysis & Kaiser Criterion

In [ ]:
# Eigenvalue analysis
fa_eigen = FactorAnalyzer(rotation=None, n_factors=len(predictor_items))
fa_eigen.fit(df_efa)
ev, v = fa_eigen.get_eigenvalues()

# Parallel analysis
from factor_analyzer.factor_analyzer import calculate_kmo

def parallel_analysis(data, n_iterations=100, percentile=95):
    """Perform parallel analysis to determine number of factors."""
    n_obs, n_vars = data.shape
    random_eigenvalues = np.zeros((n_iterations, n_vars))
    
    for i in range(n_iterations):
        random_data = np.random.normal(size=(n_obs, n_vars))
        random_corr = np.corrcoef(random_data, rowvar=False)
        random_eigenvalues[i] = np.linalg.eigvalsh(random_corr)[::-1]
    
    return np.percentile(random_eigenvalues, percentile, axis=0)

random_ev = parallel_analysis(df_efa.values)

# Determine factors by different criteria
n_factors_kaiser = sum(ev > 1)
n_factors_pa = sum(ev > random_ev)

print("=" * 90)
print("FACTOR EXTRACTION CRITERIA")
print("=" * 90)

print(f"\n{'Factor':<8} {'Eigenvalue':<12} {'Random EV':<12} {'Kaiser':<10} {'PA':<10}")
print("-" * 60)
for i in range(min(12, len(ev))):
    kaiser = "✓" if ev[i] > 1 else ""
    pa = "✓" if ev[i] > random_ev[i] else ""
    print(f"F{i+1:<7} {ev[i]:<12.3f} {random_ev[i]:<12.3f} {kaiser:<10} {pa:<10}")

print(f"\n📊 Recommendations:")
print(f"   Kaiser criterion (λ > 1): {n_factors_kaiser} factors")
print(f"   Parallel analysis: {n_factors_pa} factors")

In [ ]:
# Scree plot with parallel analysis
fig, ax = plt.subplots(figsize=(12, 6))

x = range(1, len(ev) + 1)
ax.plot(x, ev, 'bo-', linewidth=2, markersize=8, label='Actual Eigenvalues')
ax.plot(x, random_ev, 'r--', linewidth=2, label='Parallel Analysis (95th percentile)')
ax.axhline(y=1, color='gray', linestyle=':', label='Kaiser Criterion (λ=1)')

ax.set_xlabel('Factor Number', fontsize=12)
ax.set_ylabel('Eigenvalue', fontsize=12)
ax.set_title('Scree Plot with Parallel Analysis', fontsize=14)
ax.legend()
ax.set_xticks(range(1, len(ev) + 1))
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('plots/phase2_scree_plot.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Scree plot saved")

## 2.2 Full Factor Solution (All Possible Factors)

Exploring ALL factors to identify which have meaningful loadings, regardless of eigenvalue.

In [ ]:
# Fit maximum factors (up to 12 or number of items)
n_factors_max = min(12, len(predictor_items))

print(f"Fitting EFA with {n_factors_max} factors (maximum exploration)")
print(f"  Estimator: MINRES")
print(f"  Rotation: Promax (oblique)")

efa_full = FactorAnalyzer(n_factors=n_factors_max, rotation='promax', method='minres')
efa_full.fit(df_efa)

# Get variance explained
var_full = efa_full.get_factor_variance()
eigenvalues_full = var_full[0]
prop_var_full = var_full[1]
cum_var_full = var_full[2]

print(f"\n{'Factor':<8} {'SS Loading':<12} {'% Variance':<12} {'Cumulative %':<12} {'Strength':<15}")
print("-" * 70)
for i in range(n_factors_max):
    if eigenvalues_full[i] >= 1.0:
        strength = "★★★ Strong"
    elif eigenvalues_full[i] >= 0.5:
        strength = "★★ Moderate"
    elif eigenvalues_full[i] >= 0.3:
        strength = "★ Weak"
    else:
        strength = "✗ Minimal"
    print(f"F{i+1:<7} {eigenvalues_full[i]:<12.3f} {prop_var_full[i]*100:<12.1f} {cum_var_full[i]*100:<12.1f} {strength}")

print(f"\nTotal variance explained: {cum_var_full[-1]*100:.1f}%")

## 2.3 Full Pattern Matrix

In [ ]:
# Pattern matrix
loadings_full = pd.DataFrame(
    efa_full.loadings_,
    index=predictor_items,
    columns=[f'F{i+1}' for i in range(n_factors_max)]
)

# Add construct labels
loadings_full['Construct'] = [metadata[item]['construct_abbr'] for item in loadings_full.index]

# Highlighting function
def highlight_loadings(val):
    if abs(val) >= 0.50:
        return 'background-color: darkgreen; color: white; font-weight: bold'
    elif abs(val) >= 0.32:
        return 'background-color: goldenrod; color: white'
    return ''

print("FULL PATTERN MATRIX")
print("Green ≥ 0.50 (strong), Yellow ≥ 0.32 (moderate)")
print("=" * 90)

factor_cols = [f'F{i+1}' for i in range(n_factors_max)]
styled = loadings_full[factor_cols + ['Construct']].style.applymap(
    highlight_loadings, subset=factor_cols
).format('{:.3f}', subset=factor_cols)
display(styled)

## 2.4 Factor Quality Assessment

In [ ]:
# Analyze each factor's quality
print("=" * 90)
print("FACTOR QUALITY ANALYSIS (by loading strength)")
print("=" * 90)

factor_quality = []

for i in range(n_factors_max):
    factor_name = f'F{i+1}'
    factor_loadings = loadings_full[factor_name]
    abs_loadings = factor_loadings.abs()
    
    # Count items by loading strength
    strong_items = loadings_full[abs_loadings >= 0.50].index.tolist()
    moderate_items = loadings_full[(abs_loadings >= 0.32) & (abs_loadings < 0.50)].index.tolist()
    
    # Check cross-loading issues
    cross_loading_issues = []
    for item in strong_items:
        other_factors = [f'F{j+1}' for j in range(n_factors_max) if j != i]
        other_loadings = loadings_full.loc[item, other_factors].abs()
        if other_loadings.max() >= 0.32:
            cross_factor = other_loadings.idxmax()
            cross_loading_issues.append(f"{item}→{cross_factor}")
    
    # Get constructs loading on this factor
    constructs = [metadata[item]['construct_abbr'] for item in strong_items]
    unique_constructs = list(set(constructs))
    
    # Quality rating
    if len(strong_items) >= 2 and len(cross_loading_issues) == 0:
        quality = "★★★ RETAIN (clean, multi-item)"
    elif len(strong_items) >= 2:
        quality = "★★ RETAIN (minor cross-loading)"
    elif len(strong_items) == 1 and len(cross_loading_issues) == 0:
        quality = "★ CONSIDER (single-item, clean)"
    else:
        quality = "✗ SKIP"
    
    factor_quality.append({
        'Factor': factor_name,
        'Eigenvalue': eigenvalues_full[i],
        'Strong_Items': len(strong_items),
        'Cross_Loadings': len(cross_loading_issues),
        'Constructs': unique_constructs,
        'Quality': quality
    })
    
    print(f"\n{factor_name} (λ={eigenvalues_full[i]:.2f}): {quality}")
    if strong_items:
        print(f"   Strong items: {', '.join(strong_items)}")
        print(f"   Constructs: {', '.join(unique_constructs)}")
    if cross_loading_issues:
        print(f"   ⚠️ Cross-loadings: {', '.join(cross_loading_issues)}")

quality_df = pd.DataFrame(factor_quality)

In [ ]:
# Factor quality summary table
print("\n" + "=" * 90)
print("FACTOR QUALITY SUMMARY TABLE")
print("=" * 90)

display(quality_df)

# Recommendations
retain_factors = quality_df[quality_df['Quality'].str.contains('RETAIN')]
consider_factors = quality_df[quality_df['Quality'].str.contains('CONSIDER')]

print(f"\n📊 Factor Recommendations:")
print(f"   ★★★/★★ RETAIN: {len(retain_factors)} factors - {list(retain_factors['Factor'])}")
print(f"   ★ CONSIDER: {len(consider_factors)} factors - {list(consider_factors['Factor'])}")

## 2.5 Construct-to-Factor Mapping

In [ ]:
# Show which factors each construct's items load on
print("=" * 90)
print("CONSTRUCT-TO-FACTOR MAPPING")
print("=" * 90)
print("Where do items from each theoretical construct load?")

for construct in loadings_full['Construct'].unique():
    construct_items = loadings_full[loadings_full['Construct'] == construct]
    print(f"\n{construct}:")
    for item in construct_items.index:
        item_loadings = loadings_full.loc[item, factor_cols]
        max_factor = item_loadings.abs().idxmax()
        max_loading = item_loadings[max_factor]
        
        # Find any cross-loadings
        sorted_loadings = item_loadings.abs().sort_values(ascending=False)
        secondary = sorted_loadings.index[1] if len(sorted_loadings) > 1 else None
        secondary_loading = item_loadings[secondary] if secondary else 0
        
        cross = f" (also {secondary}: {secondary_loading:+.2f})" if abs(secondary_loading) >= 0.32 else ""
        print(f"   {item} → {max_factor} ({max_loading:+.3f}){cross}")

# ═══════════════════════════════════════════════════════════════════════════════
# 🎯 DECISION POINT 2: SELECT FACTORS TO RETAIN
# ═══════════════════════════════════════════════════════════════════════════════

Based on the factor exploration above, decide which factors to retain.

**Considerations:**
- Factors with strong loadings (≥ 0.50) on multiple items are preferred
- Clean factors (no cross-loadings ≥ 0.32) are easier to interpret
- Single-item factors are acceptable if theoretically meaningful
- Consider theoretical interpretability (do the items make sense together?)

**Edit the `SELECTED_FACTORS` list below to customize your selection.**

In [ ]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  🎯 DECISION POINT 2: EDIT THIS LIST TO SELECT FACTORS                       ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

# Based on the factor quality analysis above, select which factors to retain
# Example: SELECTED_FACTORS = ['F1', 'F2', 'F3', 'F6']

SELECTED_FACTORS = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6']  # <-- EDIT THIS LIST

print("=" * 90)
print("🎯 DECISION POINT 2 - FACTOR SELECTION")
print("=" * 90)
print(f"\nSelected factors: {SELECTED_FACTORS}")
print(f"Number of factors: {len(SELECTED_FACTORS)}")

# ═══════════════════════════════════════════════════════════════════════════════
# PHASE 3: FINAL MODEL & VALIDATION
# ═══════════════════════════════════════════════════════════════════════════════

## 3.1 Build Cherry-Picked Factor Solution

In [ ]:
# Build selected factor solution
selected_loadings = loadings_full[SELECTED_FACTORS + ['Construct']].copy()

# Assign items to factors
item_assignments = []
for item in selected_loadings.index:
    item_loadings = selected_loadings.loc[item, SELECTED_FACTORS]
    primary_factor = item_loadings.abs().idxmax()
    primary_loading = item_loadings[primary_factor]
    
    max_abs_loading = item_loadings.abs().max()
    if max_abs_loading >= 0.40:
        status = "✓ Assigned"
    elif max_abs_loading >= 0.32:
        status = "~ Marginal"
    else:
        status = "✗ Unassigned"
    
    item_assignments.append({
        'Item': item,
        'Construct': metadata[item]['construct_abbr'],
        'Primary_Factor': primary_factor,
        'Loading': primary_loading,
        'Status': status
    })

assignment_df = pd.DataFrame(item_assignments)

# Display by factor
print("=" * 90)
print(f"CHERRY-PICKED {len(SELECTED_FACTORS)}-FACTOR SOLUTION")
print("=" * 90)

for factor in SELECTED_FACTORS:
    factor_items = assignment_df[
        (assignment_df['Primary_Factor'] == factor) & 
        (assignment_df['Status'].isin(['✓ Assigned', '~ Marginal']))
    ]
    print(f"\n{factor}:")
    for _, row in factor_items.iterrows():
        print(f"   {row['Item']} ({row['Construct']}): {row['Loading']:+.3f} {row['Status']}")

# Unassigned items
unassigned = assignment_df[assignment_df['Status'] == '✗ Unassigned']
if len(unassigned) > 0:
    print(f"\n⚠️ UNASSIGNED ITEMS ({len(unassigned)}):")
    for _, row in unassigned.iterrows():
        print(f"   {row['Item']} ({row['Construct']}): max loading = {row['Loading']:+.3f}")

# Statistics
assigned_count = len(assignment_df[assignment_df['Status'] == '✓ Assigned'])
marginal_count = len(assignment_df[assignment_df['Status'] == '~ Marginal'])
unassigned_count = len(unassigned)

print(f"\n📊 Assignment Statistics:")
print(f"   Items assigned (≥.40): {assigned_count}/{len(predictor_items)}")
print(f"   Items marginal (.32-.40): {marginal_count}")
print(f"   Items unassigned (<.32): {unassigned_count}")

## 3.2 Factor Reliability Assessment

In [ ]:
# Calculate reliability for each factor
print("=" * 90)
print("FACTOR RELIABILITY ASSESSMENT")
print("=" * 90)

def calculate_cronbach_alpha(data):
    """Calculate Cronbach's alpha."""
    n_items = data.shape[1]
    if n_items < 2:
        return np.nan
    item_vars = data.var(axis=0, ddof=1)
    total_var = data.sum(axis=1).var(ddof=1)
    return (n_items / (n_items - 1)) * (1 - item_vars.sum() / total_var)

factor_reliability = []

for factor in SELECTED_FACTORS:
    factor_items = assignment_df[
        (assignment_df['Primary_Factor'] == factor) & 
        (assignment_df['Loading'].abs() >= 0.40)
    ]['Item'].tolist()
    
    if len(factor_items) >= 2:
        alpha = calculate_cronbach_alpha(df_efa[factor_items])
        constructs = [metadata[item]['construct_abbr'] for item in factor_items]
        primary_construct = pd.Series(constructs).mode()[0]
        status = '✓ Good' if alpha >= 0.70 else ('~ Marginal' if alpha >= 0.60 else '✗ Poor')
        
        factor_reliability.append({
            'Factor': factor,
            'N_Items': len(factor_items),
            'Items': ', '.join(factor_items),
            'Primary_Construct': primary_construct,
            'Alpha': alpha,
            'Status': status
        })
        print(f"\n{factor} ({primary_construct}): α = {alpha:.3f} {status}")
        print(f"   Items: {', '.join(factor_items)}")
    elif len(factor_items) == 1:
        factor_reliability.append({
            'Factor': factor,
            'N_Items': 1,
            'Items': factor_items[0],
            'Primary_Construct': metadata[factor_items[0]]['construct_abbr'],
            'Alpha': np.nan,
            'Status': '— Single item'
        })
        print(f"\n{factor}: Single-item ({factor_items[0]})")
    else:
        print(f"\n{factor}: No items assigned")

reliability_df = pd.DataFrame(factor_reliability)

In [ ]:
# Reliability summary table
print("\n" + "=" * 90)
print("RELIABILITY SUMMARY TABLE")
print("=" * 90)
display(reliability_df)

# Count good reliability
multi_item = reliability_df[reliability_df['N_Items'] >= 2]
good_alpha = len(multi_item[multi_item['Alpha'] >= 0.70])
total_multi = len(multi_item)

print(f"\n📊 Reliability Summary:")
print(f"   Multi-item factors: {total_multi}/{len(SELECTED_FACTORS)}")
print(f"   Good reliability (α ≥ .70): {good_alpha}/{total_multi}")
print(f"   Single-item factors: {len(reliability_df[reliability_df['N_Items'] == 1])}")

## 3.3 Pattern Matrix Visualization

In [ ]:
# Heatmap of selected factor loadings
plt.figure(figsize=(12, max(8, len(predictor_items) * 0.4)))

heatmap_data = selected_loadings[SELECTED_FACTORS].copy()

# Add construct grouping
heatmap_data['Construct'] = selected_loadings['Construct']
heatmap_data = heatmap_data.sort_values('Construct')

sns.heatmap(heatmap_data[SELECTED_FACTORS], 
            annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            vmin=-1, vmax=1,
            yticklabels=[f"{idx} ({heatmap_data.loc[idx, 'Construct']})" for idx in heatmap_data.index])

plt.title(f'Cherry-Picked {len(SELECTED_FACTORS)}-Factor Pattern Matrix\n(Items grouped by construct)', fontsize=14)
plt.xlabel('Factor')
plt.ylabel('Item (Construct)')
plt.tight_layout()
plt.savefig('plots/phase3_pattern_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Pattern matrix saved to plots/phase3_pattern_matrix.png")

## 3.4 Factor Correlation Matrix

In [ ]:
# Factor correlations (from full solution, subset to selected)
if hasattr(efa_full, 'phi_'):
    factor_corr_full = pd.DataFrame(
        efa_full.phi_,
        index=[f'F{i+1}' for i in range(n_factors_max)],
        columns=[f'F{i+1}' for i in range(n_factors_max)]
    )
    factor_corr = factor_corr_full.loc[SELECTED_FACTORS, SELECTED_FACTORS]
    
    plt.figure(figsize=(8, 6))
    mask = np.triu(np.ones_like(factor_corr, dtype=bool), k=1)
    sns.heatmap(factor_corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
               vmin=-1, vmax=1, mask=mask)
    plt.title('Factor Intercorrelations (Promax rotation)', fontsize=14)
    plt.tight_layout()
    plt.savefig('plots/phase3_factor_correlations.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✓ Factor correlation matrix saved")
else:
    print("Factor correlations not available (orthogonal rotation used)")

## 3.5 Export Results

In [ ]:
# Export all results
print("=" * 90)
print("EXPORTING RESULTS")
print("=" * 90)

# 1. Pattern matrix
loadings_export = selected_loadings.copy()
loadings_export.to_csv('tables/pattern_matrix.csv')
print("✓ tables/pattern_matrix.csv")

# 2. Item assignments
assignment_df.to_csv('tables/item_assignments.csv', index=False)
print("✓ tables/item_assignments.csv")

# 3. Factor reliability
reliability_df.to_csv('tables/factor_reliability.csv', index=False)
print("✓ tables/factor_reliability.csv")

# 4. Item diagnostics
item_diag_df.to_csv('tables/item_diagnostics.csv', index=False)
print("✓ tables/item_diagnostics.csv")

# 5. Full loadings for reference
loadings_full.to_csv('tables/full_pattern_matrix.csv')
print("✓ tables/full_pattern_matrix.csv")

# 6. JSON summary
summary = {
    'population': POPULATION,
    'n_observations': len(df_full),
    'n_items_selected': len(predictor_items),
    'n_factors_selected': len(SELECTED_FACTORS),
    'selected_items': SELECTED_ITEMS,
    'selected_factors': SELECTED_FACTORS,
    'kmo': float(kmo_model),
    'bartlett_chi2': float(chi_square),
    'variance_explained': float(cum_var_full[len(SELECTED_FACTORS)-1]) if len(SELECTED_FACTORS) <= len(cum_var_full) else None
}

with open('tables/efa_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)
print("✓ tables/efa_summary.json")

## 3.6 APA Summary

In [ ]:
# Generate APA-formatted summary
n_obs = len(df_full)
n_items = len(predictor_items)
n_factors_final = len(SELECTED_FACTORS)

# Calculate total variance for selected factors
total_var_selected = sum([eigenvalues_full[int(f[1:])-1] for f in SELECTED_FACTORS])
total_var_all = sum(eigenvalues_full)
var_pct = (total_var_selected / total_var_all) * 100 if total_var_all > 0 else 0

apa_summary = f"""
## APA-Formatted Method Summary

### Exploratory Factor Analysis - {POPULATION.title()} Sample

An exploratory factor analysis (EFA) was conducted on {n_items} predictor items using the 
full sample (*N* = {n_obs}). Prior to analysis, the suitability of the data for factor 
analysis was assessed. The Kaiser-Meyer-Olkin measure of sampling adequacy was {kmo_model:.2f}, 
indicating {kmo_interpretation.lower()} factorability (Kaiser, 1974). Bartlett's test of sphericity 
was significant, χ²({n_items * (n_items - 1) // 2}) = {chi_square:.2f}, *p* < .001, indicating that 
correlations between items were sufficiently large for factor analysis.

Factor extraction was performed using the MINRES method with oblique rotation (Promax). 
Initial parallel analysis suggested {n_factors_pa} factors, while the Kaiser criterion 
suggested {n_factors_kaiser} factors. After examining the full {n_factors_max}-factor solution, 
a {n_factors_final}-factor solution was selected based on loading quality, theoretical 
interpretability, and construct coherence.

The {n_factors_final}-factor solution explained approximately {var_pct:.1f}% of the extracted 
variance. Items were retained if they demonstrated a primary loading ≥ .40 on their assigned 
factor. Of the {n_items} items analyzed, {assigned_count} were assigned to factors with 
acceptable loadings.

### Factor Reliability

"""

# Add reliability info
for _, row in reliability_df.iterrows():
    if row['N_Items'] >= 2:
        apa_summary += f"- **{row['Factor']}** ({row['Primary_Construct']}): α = {row['Alpha']:.2f}, {row['N_Items']} items\n"
    else:
        apa_summary += f"- **{row['Factor']}** ({row['Primary_Construct']}): Single-item factor\n"

display(Markdown(apa_summary))

## 3.7 Final Conclusions

In [ ]:
# Final synthesis
conclusions = f"""
## Final Conclusions

### Model Summary
- **Sample**: {POPULATION.title()} (N = {n_obs})
- **Items Analyzed**: {n_items} (from {constructs_active} constructs)
- **Factors Retained**: {n_factors_final}
- **Items Assigned**: {assigned_count}/{n_items} ({assigned_count/n_items*100:.0f}%)

### Decision Point 1 Summary (Item Selection)
- Started with {len(ALL_PREDICTOR_ITEMS)} items across 12 constructs
- Selected {n_items} items from {constructs_active} constructs
- Constructs with full representation (2 items): {constructs_full}
- Constructs with partial representation (1 item): {constructs_partial}
- Constructs dropped: {constructs_dropped}

### Decision Point 2 Summary (Factor Selection)
- Maximum factors explored: {n_factors_max}
- Parallel analysis suggested: {n_factors_pa} factors
- Kaiser criterion suggested: {n_factors_kaiser} factors
- **Final selection**: {n_factors_final} factors ({', '.join(SELECTED_FACTORS)})

### Reliability Assessment
"""

good_rel = len(reliability_df[(reliability_df['N_Items'] >= 2) & (reliability_df['Alpha'] >= 0.70)])
marginal_rel = len(reliability_df[(reliability_df['N_Items'] >= 2) & (reliability_df['Alpha'] >= 0.60) & (reliability_df['Alpha'] < 0.70)])
poor_rel = len(reliability_df[(reliability_df['N_Items'] >= 2) & (reliability_df['Alpha'] < 0.60)])
single_item = len(reliability_df[reliability_df['N_Items'] == 1])

conclusions += f"""
- Good reliability (α ≥ .70): {good_rel} factors
- Marginal reliability (.60 ≤ α < .70): {marginal_rel} factors
- Poor reliability (α < .60): {poor_rel} factors
- Single-item factors: {single_item}

### Recommendations
"""

if unassigned_count > 0:
    conclusions += f"- ⚠️ {unassigned_count} items did not load adequately on selected factors\n"
if poor_rel > 0:
    conclusions += f"- ⚠️ {poor_rel} factors have poor internal consistency\n"
if single_item > n_factors_final // 2:
    conclusions += f"- ⚠️ Many single-item factors - consider combining or re-evaluating\n"

conclusions += """
### Next Steps
1. Validate this structure with CFA on an independent sample
2. Consider theoretical justification for factor interpretations
3. Document any items that behave unexpectedly
"""

display(Markdown(conclusions))

print("\n" + "=" * 90)
print("✓ ANALYSIS COMPLETE")
print("=" * 90)

In [ ]:
# Final synthesis
conclusions = f"""
## Final Conclusions

### Model Summary
- **Sample**: {POPULATION.title()} (N = {n_obs})
- **Items Analyzed**: {n_items} (from {constructs_active} constructs)
- **Factors Retained**: {n_factors_final}
- **Items Assigned**: {assigned_count}/{n_items} ({assigned_count/n_items*100:.0f}%)

### Decision Point 1 Summary (Item Selection)
- Started with {len(ALL_PREDICTOR_ITEMS)} items across 12 constructs
- Selected {n_items} items from {constructs_active} constructs
- Constructs with full representation (2 items): {constructs_full}
- Constructs with partial representation (1 item): {constructs_partial}
- Constructs dropped: {constructs_dropped}

### Decision Point 2 Summary (Factor Selection)
- Maximum factors explored: {n_factors_max}
- Parallel analysis suggested: {n_factors_pa} factors
- Kaiser criterion suggested: {n_factors_kaiser} factors
- **Final selection**: {n_factors_final} factors ({', '.join(SELECTED_FACTORS)})

### Reliability Assessment
"""

good_rel = len(reliability_df[(reliability_df['N_Items'] >= 2) & (reliability_df['Alpha'] >= 0.70)])
marginal_rel = len(reliability_df[(reliability_df['N_Items'] >= 2) & (reliability_df['Alpha'] >= 0.60) & (reliability_df['Alpha'] < 0.70)])
poor_rel = len(reliability_df[(reliability_df['N_Items'] >= 2) & (reliability_df['Alpha'] < 0.60)])
single_item = len(reliability_df[reliability_df['N_Items'] == 1])

conclusions += f"""
- Good reliability (α ≥ .70): {good_rel} factors
- Marginal reliability (.60 ≤ α < .70): {marginal_rel} factors
- Poor reliability (α < .60): {poor_rel} factors
- Single-item factors: {single_item}

### Recommendations
"""

if unassigned_count > 0:
    conclusions += f"- ⚠️ {unassigned_count} items did not load adequately on selected factors\n"
if poor_rel > 0:
    conclusions += f"- ⚠️ {poor_rel} factors have poor internal consistency\n"
if single_item > n_factors_final // 2:
    conclusions += f"- ⚠️ Many single-item factors - consider combining or re-evaluating\n"

conclusions += """
### Next Steps
1. Validate this structure with CFA on an independent sample
2. Consider theoretical justification for factor interpretations
3. Document any items that behave unexpectedly
"""

display(Markdown(conclusions))

print("\n" + "=" * 90)
print("✓ ANALYSIS COMPLETE")
print("=" * 90)